In [ ]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [ ]:
# import the package
import osmosispy
from osmosispy import utils, pytypes, Coin

In [ ]:
# configure
mnemonic_key = "fat patch excite gold bubble large tunnel vote fine title hover junior advice cable ordinary column mass aunt trigger lucky hope animal abandon mansion"

## Prepare the account

to start localnet clone the [osmosis-labs/osmosis](https://github.com/osmosis-labs/osmosis) repo and run the following command

```bash
make localnet-build
make localnet-start
```

In [ ]:
# # authorize in the localnet
# network = osmosispy.Network.localnet()

# authorize in the mainnet
network = osmosispy.Network.mainnet()

trader = osmosispy.Sdk.authorize(key=mnemonic_key).with_network(network).with_config(pytypes.TxConfig())

# check the address
print(trader.address)

## Check balances of the account

In [ ]:
from osmosis_proto.proto.cosmos.bank.v1beta1 import (
  query_pb2 as bank_query,
  query_pb2_grpc as bank_query_grpc,
)

In [ ]:
stub_bank = bank_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_bank.AllBalances(
        bank_query.QueryAllBalancesRequest(address=trader.address)
    )
)

print(result["balances"])


## Query Pools

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
  query_pb2 as gamm_query,
  query_pb2_grpc as gamm_query_grpc,
)

In [ ]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

print(result["pools"])

## Query Locks

In [ ]:
from osmosis_proto.proto.osmosis.lockup import (
    query_pb2 as lockup_query,
    query_pb2_grpc as lockup_query_grpc,
)

In [ ]:
# Get account locks

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.AccountLockedDuration(
        # pass owner address string to query
        lockup_query.AccountLockedDurationRequest(owner=trader.address)
    )
)

print(result["locks"])

In [ ]:
# replace with the actual lock id
lock_id=123

In [ ]:
# Lock by ID

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.LockedByID(
        # pass lock id to query
        lockup_query.LockedRequest(lock_id=lock_id)
    )
)

print(result)

## Query Epochs

In [ ]:
from osmosis_proto.proto.osmosis.epochs import (
    query_pb2 as epochs_query,
    query_pb2_grpc as epochs_query_grpc,
)

In [ ]:
stub_epochs = epochs_query_grpc.QueryStub(trader.query.chain_channel)

In [ ]:
# Get all epochs
result = utils.deserialize(
    stub_epochs.EpochInfos(
        epochs_query.QueryEpochsInfoRequest()
    )
)

print(result)

In [ ]:
# Get current epoch
result = utils.deserialize(
    stub_epochs.CurrentEpoch(
        epochs_query.QueryCurrentEpochRequest(identifier="day")
    )
)

print(result)

## Get pools

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
    query_pb2 as gamm_query,
    query_pb2_grpc as gamm_query_grpc,
)

In [ ]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pools_resp = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

# pools need to be deserialized manually as they are Any types
pools = [utils.deserialize_pool(pool) for pool in pools_resp["pools"]]

print("pools", pools)

In [ ]:
# Get specific pool by ID
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pool_raw_data = utils.deserialize(
    stub_gamm.Pool(
        # checking this pool https://app.osmosis.zone/pool/678
        gamm_query.QueryPoolRequest(pool_id=678)
    )
)

pool = utils.deserialize_pool(pool_raw_data["pool"])

print(pool)

In [ ]:
# Get spot price
price_data = utils.deserialize(
    stub_gamm.SpotPrice(
        gamm_query.QuerySpotPriceRequest(
            pool_id=pool["id"],
            base_asset_denom=pool["pool_assets"][0]["token"]["denom"],
            quote_asset_denom=pool["pool_assets"][1]["token"]["denom"],
        )
    )
)

print(price_data["spot_price"])